In [13]:
import os
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
# load data
DATA_PATH = "/Users/musatai/masters/research/code/fl-demo/databases/raw_data"
DATA_FILE = "cardio_train.csv"

data_df = pd.read_csv(os.path.join(DATA_PATH, DATA_FILE), delimiter=";")

In [3]:
data_df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
data_df.shape

(70000, 13)

In [5]:
test_data_indexes = data_df.sample(frac=0.2, random_state=1).index
test_data_indexes.shape

(14000,)

In [6]:
data_df = data_df.drop(index=test_data_indexes)
data_df.shape

(56000, 13)

In [7]:
data_df = data_df.drop("id", axis=1)
data_df["age"] = data_df["age"] // 365
data_df['bmi'] = data_df['weight'] / (data_df['height'] / 100) ** 2
data_df = data_df.drop(["height", "weight"], axis=1)
data_df = data_df[~((data_df["ap_hi"] > 250) | (data_df["ap_lo"] > 200))]
data_df = data_df[~((data_df["ap_hi"] < 60) | (data_df["ap_lo"] < 60))]

In [8]:
target = data_df["cardio"]
data_df = data_df.drop("cardio", axis=1)

In [9]:
scaler = StandardScaler()
features = scaler.fit_transform(data_df)

In [10]:
num_features = 10
model = Sequential(
    [
        Dense(128, activation="relu", input_shape=(num_features,)),
        Dropout(0.2),
        Dense(128, activation="relu"),
        Dropout(0.2),
        Dense(64, activation="relu", input_shape=(num_features,)),
        Dropout(0.2),
        Dense(1, activation="sigmoid"),
    ]
)

In [11]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [12]:
history = model.fit(
    features,
    target,
    epochs=300, 
    batch_size=32, 
    validation_split=0.2,
)

Epoch 1/300
1374/1374 [==============================] - 2s 1ms/step - loss: 0.5608 - accuracy: 0.7249 - val_loss: 0.5514 - val_accuracy: 0.7283
Epoch 2/300
1374/1374 [==============================] - 2s 1ms/step - loss: 0.5521 - accuracy: 0.7321 - val_loss: 0.5564 - val_accuracy: 0.7307
Epoch 3/300
1374/1374 [==============================] - 2s 1ms/step - loss: 0.5502 - accuracy: 0.7322 - val_loss: 0.5530 - val_accuracy: 0.7286
Epoch 4/300
1374/1374 [==============================] - 2s 1ms/step - loss: 0.5485 - accuracy: 0.7326 - val_loss: 0.5518 - val_accuracy: 0.7262
Epoch 5/300
1374/1374 [==============================] - 2s 1ms/step - loss: 0.5482 - accuracy: 0.7340 - val_loss: 0.5508 - val_accuracy: 0.7299
Epoch 6/300
1374/1374 [==============================] - 2s 1ms/step - loss: 0.5466 - accuracy: 0.7330 - val_loss: 0.5502 - val_accuracy: 0.7294
Epoch 7/300
1374/1374 [==============================] - 2s 1ms/step - loss: 0.5459 - accuracy: 0.7343 - val_loss: 0.5544 - val_ac

In [18]:
with open("/Users/musatai/masters/research/code/fl-demo/databases/train_history/centralized/history.pickle", "wb") as fp:
    pickle.dump(history.history, fp)

In [16]:
with open("//Users/musatai/masters/research/code/fl-demo/databases/centralized_model/centralized_model.pickle", "wb") as fp:
    pickle.dump(model, fp)